# 1. Selenium 이란?

* 웹브라우저를 자동으로 제어하는 라이브러리
* 본래 다양한 웹브라우저를 사용한 웹 사이트 자동 테스트 도구
* 현재는 웹브라우저를 자동 제어하여 동적 웹사이트에서 정보를 가져오는 용도로도 많이 활용

# 2. 정적 웹 사이트 vs 동적 웹 사이트
* 정적 웹 사이트(Static website): HTML 페이지 안에 웹 브라우저에 표시되는 모든 정보가 포함된 사이트
 * requests 라이브러리를 활용해서 정보 추출 가능
* 동적 웹 사이트(Dynamic website): Java Script를 사용하여 상황에 따라 필요한 정보를 불러와 표시하는 웹 사이트
 * 상황에 맞추어 자바스크립트를 활용해 서버에 정보를 다시 요청하는 형식이라 requests로는 정보 추출 불가능
 * selenium으로 웹브라우저를 조작해 원하는 정보가 다 표시 된 후 정보 추출 필요

# 3. Selenium 웹사이트
[Selenium website](https://www.selenium.dev/documentation/webdriver/)

# 4. Selenium 설치

In [1]:
# !pip install selenium
# !pip install webdriver-manager
# !pip install lxml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


# 5. WebDriver Manager 및 WebDriver 설치
* [webdriver manager 설치](https://github.com/SergeyPirogov/webdriver_manager)

In [1]:
import selenium
print(selenium.__version__)

4.22.0


In [6]:
# selenium 4
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service as ChromiumService
# from webdriver_manager.chrome import ChromeDriverManager
# from webdriver_manager.core.utils import ChromeType

# driver = webdriver.Chrome(service=ChromiumService(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()))

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import time
import lxml

In [10]:
# 크롬 옵션즈에 User-Agent, lang 같은 정보를 담아 셀레니움을 이용한 크롤링이 아닌 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36')
options.add_argument("lang=ko_KR")

# 웹드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver.set_window_size(1920,1080)   # 웹브라우저 해상도 조절

driver.get("https://search.shopping.naver.com/book/home")  # 크롤링 할 웹사이트 주소

In [12]:
# 네이버 쇼핑에서 오른쪽 끝의 도서검색창을 찾아 검색어 입력
search_box = driver.find_element(By.CSS_SELECTOR, "#lnb > div > div._lnbSearch_lnb_search_10s9T > div > div > form > input")
search_box.send_keys("파이썬"+ Keys.ENTER)

In [16]:
# 검색되어 나온 결과는 스크롤을 아래로 내려야 한 페이지에 40개의 결과가 노출

all_bookList = []
for page in tqdm(range(1,20)):   # 반복문을 이용해 1000px씩 스크롤을 7번 내리도록 해서 전체 페이지 로딩
    y = 0
    y_step = 1000
    for i in range(1,8):
        y = y + y_step
        script = f"window.scrollTo({0},{y})"    # 자바스크립트로 위치 지정
        driver.execute_script(script)                      # 스크롤 실행
        time.sleep(5)
    page_html = driver.page_source          # 페이지 끝에서 40개의 책이 로딩된 결과를 저장
    soup = bs(page_html, "html.parser")     # beautifulsoup으로 html을 parsing
    all_bookList.append(soup)               # parsing된 결과를 all_bookList에 저장
    driver.find_element(By.CSS_SELECTOR, "#container > div > div.bookSearch_book_search__Fm5Em > div > div.Paginator_list_paging__hUKRc > button.Paginator_btn_next__7NiBL").click()  # 다음페이지 넘김 버튼 클릭


100%|██████████████████████████████████████████████████████████████████████████████████████| 19/19 [11:10<00:00, 35.31s/it]


In [17]:
driver.close()

In [18]:
len(all_bookList[0].select_one("#book_list > ul.list_book").select("li"))

40

In [21]:
all_bookList[0].select_one("#book_list > ul.list_book").select("li")

[<li class="bookListItem_item_book__X0Yfi"><div class="bookListItem_item_inner__9LmpD"><a class="bookListItem_info_top__r54Eg linkAnchor" data-i="42181812624" data-nclick="N=a:stab1l*c.info,i:42181812624,r:1" href="https://cr3.shopping.naver.com/book/adcr?x=JPCf5W0biP8Z232E0ildnP%2F%2F%2Fw%3D%3Dsi3hcu3W2Ei2zBtdJx3ZSZp%2F1Id8es2GI4aw3xtdVVDi57StUFLKmRh1TRW6ZM83N%2FUvRXxe9dkvDHkhclQYyNx4pza8ebIOIXPrPsGGS95ek6v2w9jBqMGR4TTsgYaQn4WY5w%2FYautx%2F05%2BTK2t%2FZS7bpUGcIxzH3xtrNEGqGRIrquFGLOPEkg5%2Fsh68SVnXINYqmtreNhFidWZDsxs2uVVd6Z%2BmZ5fvTALbBBPmfKIKVnJqocqfFwLra6hFr54Bk596FrdL3uz5WR1S3i83%2Ffj6xDMeJaPTHtDIoNWns%2BM%2BoN22s6dKwm7Qn7boTIbf2IasE4sOr%2FhXGsmbdiH1QUPg%2F6PYJW9LpCSumzPJoih8XCD2T3rMHrg%2B%2BhMjzimEyS4%2FLIr9OspJg7ogUQ%2BI%2B2ami%2Bb0ZTzpY0HsrBfzbf1oefHggQ1bYyBzAsIxOJAIOOvg0szObreKXcfVz48uY0q1OWp%2FVeeXkrckO%2Bh12EbJaEwWx6Kky0nst6AxUNu5HiCYyzSG0zC1gFZujw8fhjpy3bHiUzbpbWIbJKULTrE0VxTrJnZisk%2FodTU%2B2nS4gaTmsHOi4pxcTo6ldDev%2BPak6Jft9WWk4toZLgkDM4jks4qGbaANy%2Bv7%2F5bsJz2WSp0iIBffu3C

In [19]:
import pandas as pd

In [22]:
result2 = pd.DataFrame()

# for page in all_bookList:
#     for book in book_list2:
for page in all_bookList:
    for book in tqdm(page.select_one("#book_list > ul.list_book").select("li")):
        title = book.select_one("bookListItem_text__SL9m9").text
        detail_link = book.select_one(".bookListItem_item_inner__Fp7hN > a")['href']
        ranking = book.select_one("div.bookListItem_feature__txTlp").text if book.select_one("div.bookListItem_feature__txTlp") != None else "None"
        author = book.select_one("span.bookListItem_define_data__kKD8t").text if book.select_one("span.bookListItem_define_data__kKD8t") != None else "None"
        pubulisher = book.select_one("div.bookListItem_detail_publish__FgPYQ > span.bookListItem_define_data__kKD8t").text if book.select_one("div.bookListItem_detail_publish__FgPYQ > span.bookListItem_define_data__kKD8t") != None else "None"
        starpoint = book.select_one("div.bookListItem_grade__tywh2").text[2:5] if book.select_one("div.bookListItem_grade__tywh2") != None else "0"
        n_reviews = book.select_one("div.bookListItem_grade__tywh2").text[6:].strip("() ") if book.select_one("div.bookListItem_grade__tywh2") != None else "0"
        price = book.select_one("em").text if book.select_one("em") != None else "0"
        temp = dict(제목 = [title], 상세정보 = [detail_link], 랭킹 = [ranking] , 저자 = [author], 출판사 = [pubulisher],
                    평점 = [starpoint], 리뷰수 = [n_reviews], 가격 = [price])
        temp_df = pd.DataFrame(temp)
#         display(temp_df)
        result2 = pd.concat([result2, temp_df])



  0%|                                                                                               | 0/40 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
result2.reset_index(drop=True, inplace=True)

In [ ]:
result2

In [ ]:
finalData = result2[result2["저자"] != 'None']

In [ ]:
finalData.drop_duplicates(subset="제목", keep='first')

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///datas/Chicken_shops2.db', echo=False)
conn = engine.raw_connection()
Chicken_Shops.to_sql('Chicken_Shops2', con=conn, if_exists='append')

In [ ]:
test = pd.read_sql('select * from Chicken_Shops2', conn)
test